## Importing necessary modules


In [8]:
import cv2
import matplotlib.pyplot as plt
import numpy as np

In [9]:
img_src_4_points = cv2.imread("images/homography_1.jpg")
resize_parameter = 1.3
plane_size = (round(130 * resize_parameter),round(160 * resize_parameter))


## Interface to select points and input their real coordinates

In [14]:

# Define the callback function for mouse events
def mouse_callback(event, x, y, flags, params):
    # If left button is clicked, record the point coordinates
    if event == cv2.EVENT_LBUTTONDOWN:
        try:
            params['points'].append([x, y])
            cv2.circle(params['img_select_points'], (x, y), 5, (0, 0, 255), -1)
            x,y = input('Enter the real coordinates of the point in the format: x, y: ').split(',')
            params['real_coords'].append([float(x), float(y)])
        except:
            print('Invalid input')
            params['points'].pop()
            cv2.circle(params['img_select_points'], (x, y), 5, (0, 0, 0), -1)
            if len(params['real_coords']) > params['points']:
                params['real_coords'].pop()

# Create a window and set the mouse callback function
def select_points_from_image(img_src, no_of_points=4):
    img_select_points = img_src.copy()
    cv2.namedWindow('image')
    params = {'points': [], 'real_coords': [], 'img_select_points': img_select_points}
    cv2.setMouseCallback('image', mouse_callback, params)

    # Display the image and wait for user to select points
    while True:
        cv2.imshow('image', img_select_points)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        if len(params['points']) == no_of_points:
            break

    selected_pts = np.array(params['points']) 
    real_coords =  np.array(params['real_coords'])

    # Print the selected points
    print('Selected points:', selected_pts)
    print('Real coordinates:', real_coords)
    cv2.destroyAllWindows()
    return selected_pts, real_coords

pts_selected, coords_real = select_points_from_image(img_src_4_points, 4)

Selected points: [[443 197]
 [539 295]
 [369 378]
 [428 578]]
Real coordinates: [[ 20.  24.]
 [ 70.  48.]
 [ 50.  96.]
 [100. 137.]]


## Calculate homography matrix and Warp image based on it

In [16]:
def calculate_homography_and_warp(selected_pts, real_coords,img_src, plane_size, no_of_points=4):
    # Calculate Homography
    h, status = cv2.findHomography(selected_pts, real_coords)


    print('Homography Matrix:\n', h)
    # Warp source image to destination based on homography
    im_out = cv2.warpPerspective(img_src, h, plane_size)

    cv2.imwrite(f'images/warped_image_{no_of_points}_points.jpg', im_out)
    # Display images
    cv2.imshow("Source Image", img_src)
    cv2.imshow("Warped Source Image", im_out)

    cv2.waitKey(0)

    cv2.destroyAllWindows()
    return h

selected_pts = np.array(pts_selected) 
real_coords =  np.array(coords_real)
homography_matrix = calculate_homography_and_warp(selected_pts, real_coords,img_src_4_points, plane_size)

Homography Matrix:
 [[ 3.88661593e-01  4.65070366e-01 -2.35532733e+02]
 [-1.93468385e-01  6.21177176e-01 -2.74955057e+00]
 [ 2.47734758e-04  1.54017393e-03  1.00000000e+00]]


## Warp results with 4 points

### Original Image
![](./images/homography_1.jpg)

### Warped Image
![](./images/warped_image_4_points.jpg)

## Select two points on the image and calculate real distance and real coords on plane

In [13]:

def point_select_and_dist(img_select_points, h):
    
    def midpoint(p1, p2):
        return (round((p1[0]+p2[0])/2), round((p1[1]+p2[1])/2))

    def mouse_callback(event, x, y, flags, params):
        # If left button is clicked, record the point coordinates
        if event == cv2.EVENT_LBUTTONDOWN:
            point_in_image = np.array([x, y])
            params['image_coords'].append(point_in_image)
            # Convert the point to homogeneous coordinates
            point_hom = np.append(point_in_image, 1)
            # Compute the corresponding point in planar coordinates
            point_in_planar = np.dot(h, point_hom)
            point_in_planar /= point_in_planar[2] # Normalize the point  
            point_in_planar = point_in_planar[:2] # Remove the third coordinate
            params['planar_coords'].append(point_in_planar)
            cv2.circle(img_select_points, (x, y), 5, (255, 0, 0), -1)
            cv2.putText(img_select_points, str(point_in_planar), (x + 1, y + 1), cv2.FONT_HERSHEY_PLAIN, 1, (255, 0, 0), 2, cv2.LINE_AA)

    # Create a window and set the mouse callback function
    cv2.namedWindow('image')
    params = {'image_coords': [], 'planar_coords': []}
    cv2.setMouseCallback('image', mouse_callback, params)

    # Display the image and wait for user to select points
    while True:
        cv2.imshow('image', img_select_points)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        if len(params['image_coords']) == 2:
            break

    cv2.destroyWindow('image')
    # Print the selected points
    print('Point 1:', params['image_coords'][0], '->', params['planar_coords'][0])
    print('Point 2:', params['image_coords'][1], '->', params['planar_coords'][1])
    distance = np.linalg.norm(params['planar_coords'][0] - params['planar_coords'][1])
    print('Euclidean Distance between points :', distance, ' mm')

    # draw a line in between the two points
    cv2.line(img_select_points, tuple(params['image_coords'][0]), tuple(params['image_coords'][1]), (0, 255, 0), 2)
    middle_point_of_line = midpoint(params['image_coords'][0], params['image_coords'][1])
    cv2.putText(img_select_points, str(distance), middle_point_of_line, cv2.FONT_HERSHEY_PLAIN, 1, (0, 0, 0), 2, cv2.LINE_8)

    cv2.imshow('image_with_dist', img_select_points)
    cv2.imwrite('images/line_with_dist_4_points.jpg', img_select_points)
    cv2.waitKey(0)

    # Close all windows
    cv2.destroyAllWindows()

img_select_points = img_src_4_points.copy()
point_select_and_dist(img_select_points,homography_matrix)

Point 1: [442 198] -> [20.04058369 24.55908067]
Point 2: [428 579] -> [ 99.97359462 137.32047374]
Euclidean Distance between points : 138.21873246318984  mm


# Image with distance between two points
Its quite accurate! 

![ayo](./images/line_with_dist_4_points.jpg)

## Introducing the same plane but now with 10 points

In [ ]:
img_src_10_points = cv2.imread("images/homography_3.jpg")

pts_selected_10, coords_real_10 = select_points_from_image(img_src_10_points,10)


## Calculate homography matrix and Warp image based on it (again)

In [ ]:
selected_pts_10 = np.array(pts_selected_10) 
real_coords_10 =  np.array(coords_real_10)
homography_matrix = calculate_homography_and_warp(selected_pts_10, real_coords_10,img_src_10_points, plane_size, 10)

## Warp results with 10 points

### Original Image
![](./images/homography_3.jpg)

### Warped Image
![](./images/warped_image_10_points.jpg)

In [ ]:
img_select_points = img_src_10_points.copy()
point_select_and_dist(img_select_points,homography_matrix)